Hacky as but will do

In [12]:
import pyam
import pandas as pd
import numpy as np

In [13]:
EXAMPLE_FILE = "ceds-format-example-2.csv"

In [14]:
sdf = pyam.IamDataFrame(data=EXAMPLE_FILE)

INFO:root:Reading `ceds-format-example-2.csv`


Salt the data by adjusting all values randomly between 0.5 and 1.5 times the original values.

In [15]:
sdf.data.value = sdf.data.value * (1.5 - np.random.rand(len(sdf.data)))

In [16]:
sdf.head()

,model,scenario,region,variable,unit,year,value
0,foo,bar,R5ASIA,Emissions|BC,Mt BC/yr,2015,6.718883
91,foo,bar,R5LAM,Emissions|BC,Mt BC/yr,2015,0.970239
174,foo,bar,R5MAF,Emissions|BC,Mt BC/yr,2015,3.280715
257,foo,bar,R5OECD,Emissions|BC,Mt BC/yr,2015,0.855835
348,foo,bar,R5REF,Emissions|BC,Mt BC/yr,2015,0.459753


In [17]:
wdf = pd.DataFrame()
for variable in sdf.variables():
    region_var = ((sdf.data.variable == variable) & (sdf.data.region != 'World')).any()
    if not region_var:
        wdf = pd.concat([wdf, sdf.data[sdf.data.variable == variable]])
        
wdf = pyam.IamDataFrame(data=wdf)

In [18]:
rsdf = sdf.filter(
    level=2,
).filter(
    region="World",
    keep=False
)

In [19]:
bvdf = rsdf.data.copy()
bvdf["base_variable"] = bvdf["variable"].apply(lambda x: "|".join(x.split("|")[:-1]))

rsadf = bvdf.groupby(["model", "scenario", "region", "unit", "base_variable", "year"]).sum()
rsadf = rsadf.reset_index()
rsadf = rsadf.rename(columns={"base_variable": "variable"})

rsadf = pyam.IamDataFrame(rsadf)

In [20]:
wsadf = rsdf.data.groupby(["model", "scenario", "unit", "variable", "year"]).sum().reset_index()
wsadf["region"] = "World"
wsadf = pyam.IamDataFrame(wsadf)

In [21]:
avdf = sdf.filter(variable="Emissions|*|Aircraft").data
shipdf = sdf.filter(variable="Emissions|*|International Shipping").data

wbvdf = pd.concat([wsadf.data, avdf, shipdf], sort=True)
wbvdf["base_variable"] = wbvdf["variable"].apply(lambda x: "|".join(x.split("|")[:-1]))

wadf = wbvdf.groupby(["model", "scenario", "region", "unit", "base_variable", "year"]).sum()
wadf = wadf.reset_index()
wadf = wadf.rename(columns={"base_variable": "variable"})

wadf = pyam.IamDataFrame(wadf)

In [22]:
odf = pd.concat([wdf.data, rsdf.data, rsadf.data, wsadf.data, wadf.data], sort=True)
odf = odf[["model", "scenario", "variable", "region", "unit", "year", "value"]]
odf.sort_values(["model", "scenario", "variable", "region", "unit", "year", "value"])
odf = pyam.IamDataFrame(odf)

# can use this once released for final check
#odf.check_internal_consistency()

In [23]:
odf.variables()

0                                           Emissions|BC
1                Emissions|BC|Agricultural Waste Burning
2                                  Emissions|BC|Aircraft
3                             Emissions|BC|Energy Sector
4                            Emissions|BC|Forest Burning
5                         Emissions|BC|Grassland Burning
6                         Emissions|BC|Industrial Sector
7                    Emissions|BC|International Shipping
8                              Emissions|BC|Peat Burning
9              Emissions|BC|Residential Commercial Other
10                    Emissions|BC|Transportation Sector
11                                    Emissions|BC|Waste
12                                        Emissions|C2F6
13                                        Emissions|CCl4
14                                         Emissions|CF4
15                                      Emissions|CFC-11
16                                     Emissions|CFC-113
17                             

In [24]:
odf.regions()

0    R5ASIA
1     R5LAM
2     R5MAF
3    R5OECD
4     R5REF
5     World
Name: region, dtype: object

In [25]:
odf[["variable", "region"]].drop_duplicates()

,variable,region
0,Emissions|BC,R5ASIA
90,Emissions|BC,R5LAM
180,Emissions|BC,R5MAF
270,Emissions|BC,R5OECD
360,Emissions|BC,R5REF
0,Emissions|BC,World
1,Emissions|BC|Agricultural Waste Burning,R5ASIA
92,Emissions|BC|Agricultural Waste Burning,R5LAM
175,Emissions|BC|Agricultural Waste Burning,R5MAF
258,Emissions|BC|Agricultural Waste Burning,R5OECD


In [26]:
odf.head()

,model,scenario,variable,region,unit,year,value
0,foo,bar,Emissions|BC,R5ASIA,Mt BC/yr,2015,4.818396
90,foo,bar,Emissions|BC,R5LAM,Mt BC/yr,2015,0.969413
180,foo,bar,Emissions|BC,R5MAF,Mt BC/yr,2015,2.170508
270,foo,bar,Emissions|BC,R5OECD,Mt BC/yr,2015,0.818853
360,foo,bar,Emissions|BC,R5REF,Mt BC/yr,2015,0.459987


In [27]:
odf.to_csv(EXAMPLE_FILE, index=False)